In [1]:
#The necessary imports
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random
import copy
import os
from tqdm import tqdm

# Train dataset creation

In [3]:
#The file that we want to perform ER on
input_path='jf130623/ftfever_0_feveroustrainnb_0_bartlarge_5000_pos_bartbase-WikiFactEnglish_5000.jsonl'
f=open(input_path)
obj=[]
for line in f:
    obj+=[json.loads(line)]
f.close()

In [4]:
res=[x for x in obj if x['label']=='SUPPORTS']

In [5]:
resneg=[x for x in obj if x['label']=='REFUTES']

# ER generation

In [25]:
#The model that we will use to perform ER
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

print(model.device)

cuda:0


In [27]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [ ]:
#If we generate the negative from the Supports examples, set to True
pos_same_seed_neg=True

In [28]:
obj2=res if pos_same_seed_neg else resneg

In [30]:
obj2_fake=[]
i=0
for elt_h in tqdm(obj2):
    i+=1
    elt=copy.deepcopy(elt_h)
    tokens=list(enumerate(preprocess(elt['claim'])))
    tokens_NN=[x for x in tokens if len(x[1][0])>3 or x[1][0].isdigit()]#if x[1][1] in ['NN','NNP','NNS']]
    if len(tokens_NN)==0:
        print('problem')
        continue
    gen_words=[]
    tokens_NN_shuffled=copy.deepcopy(tokens_NN)
    random.shuffle(tokens_NN_shuffled)
    for elt2 in tokens_NN_shuffled :
        if len(gen_words)>3:
            break
        ####TO NOT GEN TOO MUCH USELESS FLANT5 ALTERNATIVES
        or_word=elt2[1][0]
        input_text = "Answer the following question by giving me antonyms. Can you give me an antonym of "+or_word+"?"
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
        outputs = model.generate(input_ids)
        gen_word=tokenizer.decode(outputs[0]).replace('<pad> ','').replace('</s>','')
        if len(gen_word)>0 and not or_word.lower() in gen_word.lower(): 
            gen_words+=[[or_word,gen_word]]
    if len(gen_words)==0:
        print('##########problem')
        continue
    #print(gen_words)
    elt['original_claim']=elt['claim']
    claim=elt['claim'].replace('\n','')
    for elt2 in gen_words:
        claim=claim.replace(elt2[0],elt2[1])
        
    #print(claim)
    elt['claim']=claim
    elt['label']='REFUTES'
    obj2_fake+=[elt]

  0%|          | 0/5000 [00:00<?, ?it/s]/home//.conda/envs/feverous2/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 19/5000 [00:28<1:26:47,  1.05s/it]

##########problem


  1%|          | 40/5000 [01:01<1:28:24,  1.07s/it]

##########problem


  2%|▏         | 82/5000 [02:01<2:10:25,  1.59s/it]

...

 99%|█████████▉| 4954/5000 [1:57:59<00:38,  1.20it/s]

##########problem


 99%|█████████▉| 4961/5000 [1:58:12<01:04,  1.64s/it]

##########problem


100%|██████████| 5000/5000 [1:59:10<00:00,  1.43s/it]


In [31]:
#The output folder for our experiment
folder='exp_er_fromrandom_noft_fromneg_txt_021123'


#The name for our positive model in the output file
pos_name='bartlarge NOFT'
#The name we want to use for negative model in output file
neg_name='bartlarge NOFT_er_neg'




if not os.path.exists(folder):
    os.mkdir(folder)
  
    
res=obj2+obj2_fake
params_to_write=dict()


params_to_write['pos_len']=len([x for x in res if x['label']=='SUPPORTS'])
params_to_write['total_len']=params_to_write['pos_len'] if pos_same_seed_neg else len(res)
params_to_write['neg_len']=len([x for x in res if x['label']=='REFUTES'])

params_to_write['pos_model']=pos_name
params_to_write['neg_model']=neg_name


base_fn=pos_name+'_'+str(params_to_write['pos_len'])+'_pos_'+neg_name+'_'+str(params_to_write['neg_len'])+'_neg'


f4=open(folder+'/'+base_fn+'_params.json','w')
json.dump(params_to_write,f4)

f4.close()


f4=open(folder+'/'+base_fn+'.jsonl','w')
for elt in res:
    f4.write(json.dumps(elt)+'\n')

f4.close()